# **Draft**

In [111]:
# nn stuff
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
# helper stuff
import numpy as np
import math
import h5py
import matplotlib.pyplot as plt
import scipy

In [21]:
"""Variables"""
# names
DATA = "dataset.hdf5"
x_GroundTruth = [] # this is our Y
P_SamplingOperator = [] # [] is how u initialize array (to nothing)
x_hat_ZeroFilledImg = [] # this is our x (input to NN)
y_NoisyMeasurement = []
e = 0
"""dividing data"""
data_total = 360 # got 360 images total
# training part
train_total = 300
train_ct = 294
validate_ct = 6 # train_ct + validate_ct = train_total
dev_ct = 32 # for development
# testing part
test_total = 60
test_ct = 6 # test_ct = (1/10)test_total


In [22]:
# load data
with h5py.File(DATA, 'r') as hdf:
    keys = list(hdf.keys())
    trnOrg = hdf.get('trnOrg')
    x_GroundTruth = np.array(trnOrg)
    trnMask = hdf.get('trnMask')
    P_SamplingOperator = np.array(trnMask)

In [23]:
"""calculate y"""
# Formula: y = P*Fourier(x) + e
y_NoisyMeasurement = P_SamplingOperator * np.fft.fft2(x_GroundTruth) + e
"""calculate x_hat"""
# Formula: x_hat = InverseFourier(y)
x_hat_ZeroFilledImg = np.fft.ifft2(y_NoisyMeasurement)

In [24]:
"""divide data to: training & testing set"""
# Reference: cs231n, Assignment 1, svm.ipynb
train_SET_mask = range(train_total)
X_SET_TRAIN = x_hat_ZeroFilledImg[train_SET_mask]
Y_SET_TRAIN = x_GroundTruth[train_SET_mask]

test_SET_mask = range(train_total, train_total + test_total)
X_SET_TEST = x_hat_ZeroFilledImg[test_SET_mask]
Y_SET_TEST = x_GroundTruth[test_SET_mask]

train_mask = range(train_ct)
x_train = X_SET_TRAIN[train_mask]
y_train = Y_SET_TRAIN[train_mask]

validate_mask = range(train_ct, train_ct + validate_ct)
x_val = X_SET_TRAIN[validate_mask]
y_val = Y_SET_TRAIN[validate_mask]

dev_mask = np.random.choice(train_ct, dev_ct, replace=False)
x_dev = X_SET_TRAIN[dev_mask]
y_dev = Y_SET_TRAIN[dev_mask]

test_mask = range(test_ct)
x_test = X_SET_TEST[test_mask]
y_test = Y_SET_TEST[test_mask]

In [29]:
"""preprocess"""
# turn each image into rows
x_train = np.reshape(x_train, (x_train.shape[0], -1))
x_val = np.reshape(x_val, (x_val.shape[0], -1))
x_dev = np.reshape(x_dev, (x_dev.shape[0], -1))
x_test = np.reshape(x_test, (x_test.shape[0], -1))
# preprocess: subtract mean
mean_img = np.mean(x_train, axis=0)
x_train -= mean_img
x_val -= mean_img
x_dev -= mean_img
x_test -= mean_img

### **U-Net description**
Source: https://arxiv.org/pdf/1505.04597.pdf
<br>It consists of a contracting path (left side) and an expansive path (right side). 

The contracting path follows the typical architecture of a convolutional network. 
>It consists of the repeated application of two 3x3 convolutions (unpadded convolutions), 
><br>each followed by a rectified linear unit (ReLU) 
><br>and a 2x2 max pooling operation with stride 2 for downsampling. 
><br>At each downsampling step we double the number of feature channels. 

Every step in the expansive path consists of 
>an upsampling of the feature map followed by a 2x2 convolution (“up-convolution”) that halves the number of feature channels, 
><br>a concatenation with the correspondingly cropped feature map from the contracting path, 
><br>and two 3x3 convolutions, each followed by a ReLU. 
><br>The cropping is necessary due to the loss of border pixels in every convolution. 

At the final layer a 1x1 convolution is used to map each 64-component feature vector to the desired number of classes. In total the network has 23 convolutional layers.

In [140]:
"""Encapsulated: two convolutions, each followed by ReLU"""
class Conv3x3_Relu_2x(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.go = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=0),
            nn.ReLU()
        )
    def forward(self, x):
        return self.go(x)

def Concatenate(big, small):
    big_len = big.size()[0]
    small_len = small.size()[0]
    start = big_len//2 - small_len//2
    end = start + small_len
    big = big[start:end, start:end]
    return torch.cat([big, small], dim=1)

In [144]:
"""Neural Network"""
# Reference:
# https://arxiv.org/pdf/1505.04597.pdf
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

class Me_Net(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(Me_Net, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        # (?) No need flatten, because I did it in preprocessing
        # (?) cannot use custom function inside nn.Sequential ;-;
        # two 3x3 convolutions, unpadded, each followed by rectified linear unit
        # in_channels = 2, because MRI image has real and complex parts
        # (?) out_channels = 2, because output and input images have same format
        # Question: stride = 1 ?
        # (?) no padding?
        self.Contract_1 = (Conv3x3_Relu_2x(n_channels, 64))
        # followed by 2x2 max pooling, stride = 2, for downsampling
        self.c1_c2 = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.Contract_2 = (Conv3x3_Relu_2x(64, 128))
        self.c2_c3 = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.Contract_3 = (Conv3x3_Relu_2x(128, 256))
        self.c3_c4 = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.Contract_4 = (Conv3x3_Relu_2x(256, 512))
        self.c4_c5 = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.Bottom = (Conv3x3_Relu_2x(512, 1024))
        # upsampling of the feature map,
        # followed by a 2x2 (up-)convolution that halves the feature channels 
        # (?) transpose convolution OR upsampling
        # seems like transpose convolution because I can't find way to reduce depth with upsampling
        # (?) how many steps for transpose convolution?
        # seems like 2, because in contracting step, 2x2 filter + stride 2 = 1/2 image dimensions
        # (?) a concatenation with the correspondingly cropped feature map from the contracting path
        # and two 3x3 convolutions, each followed by a ReLU
        # (?) align_corners = False
        # Reference: https://github.com/pytorch/vision/issues/1708
        self.c5_c4 = nn.ConvTranspose2d(1024, 512, kernel_size=(2,2), stride=2)
        self.Expand_4 = (Conv3x3_Relu_2x(1024, 512))
        self.c4_c3 = nn.ConvTranspose2d(512, 256, kernel_size=(2,2), stride=2)
        self.Expand_3 = (Conv3x3_Relu_2x(512, 256))
        self.c3_c2 = nn.ConvTranspose2d(256, 128, kernel_size=(2,2), stride=2)
        self.Expand_2 = (Conv3x3_Relu_2x(512, 128))
        self.c2_c1 = nn.ConvTranspose2d(128, 64, kernel_size=(2,2), stride=2)
        self.Expand_1 = (Conv3x3_Relu_2x(128, 64))
        self.out = nn.Conv2d(in_channels=64, out_channels=2, kernel_size=(1, 1), stride=1)
    def forward(self, x):
        Cont_1 = self.Contract_1(x) # take 1/2 to concatenate
        c1_c2 = self.c1_c2(Cont_1)
        Cont_2 = self.Contract_2(c1_c2) # take 1/2 to concatenate
        c2_c3 = self.c2_c3(Cont_2)
        Cont_3 = self.Contract_3(c2_c3) # take 1/2 to concatenate
        c3_c4 = self.c3_c4(Cont_3)
        Cont_4 = self.Contract_4(c3_c4) # take 1/2 to concatenate
        c4_c5 = self.c4_c5(Cont_4)
        bottom = self.Bottom(c4_c5)
        c5_c4_half = self.c5_c4(bottom) # add other 1/2
        c5_c4_full = Concatenate(Cont_4, c5_c4_half)
        Expa_4 = self.Expand_4(c5_c4_full) 
        c4_c3_half = self.c4_c3(Expa_4) # add other 1/2
        c4_c3_full = Concatenate(Cont_3, c4_c3_half)
        Expa_3 = self.Expand_3(c4_c3_full)
        c3_c2_half = self.c3_c2(Expa_3) # add other 1/2
        c3_c2_full = Concatenate(Cont_2, c3_c2_half)
        Expa_2 = self.Expand_2(c3_c2_full)
        c2_c1_half = self.c3_c2(Expa_2) # add other 1/2
        c2_c1_full = Concatenate(Cont_1, c2_c1_half)
        Expa_1 = self.Expand_1(c2_c1_full)
        logits = self.out(Expa_1)
        return logits

model = Me_Net(2,2).to(device)

Using cpu device
